In [42]:
import requests
import pandas as pd
import numpy as np
import re
from time import sleep
import time

In [43]:
data = requests.get('https://www.totalcorner.com/match/today')

In [44]:
df = pd.read_html(data.text)[0]

In [45]:
df.dropna(axis=1,how='all',inplace=True)

In [46]:
df.dropna(axis=0,how='any',inplace=True)

In [47]:
df.drop(columns=['Handicap','Goal Line','Analysis','Time'],inplace=True)

In [48]:
df.rename(columns={'Unnamed: 3':'Cronometro'},inplace=True)

In [49]:
df['Cronometro'] = df['Cronometro'].apply(lambda a: 45 if a == 'Half' else a).astype('int64')

In [50]:
df['Corner'] = df['Corner'].apply(lambda a: re.findall(r'\d{1,} - \d{1,}',a))

In [51]:
df['Corner'] = df['Corner'].apply(lambda a: list(map(int,a[0].split(' - '))))

In [52]:
df['Dangerous Attack'] = df['Dangerous Attack'].apply(lambda a: re.findall(r'\d{1,} - \d{1,}',a))

In [53]:
df['Shots'] = df['Shots'].apply(lambda a: re.findall(r'\d{1,} - \d{1,}',a))

In [54]:
df['Dangerous Attack'] = df['Dangerous Attack'].apply(lambda a: a if a != [] else np.nan)
df['Corner'] = df['Corner'].apply(lambda a: a if a != [] else np.nan)
df['Shots'] = df['Shots'].apply(lambda a: a if a != [] else np.nan)

In [55]:
df.dropna(axis=0, how = 'any',inplace=True)

In [56]:
df['Shots'] = df['Shots'].apply(lambda a: list(map(int,a[0].split(' - '))))
df['Dangerous Attack'] = df['Dangerous Attack'].apply(lambda a: list(map(int,a[0].split(' - '))))

In [57]:
df.head()

,League,Cronometro,Home,Score,Away,Corner,Dangerous Attack,Shots
2,Ecuador Campeonato Nacional,78,2 Barcelona Guayaquil,3 - 1,Cumbaya FC 2 1,"[2, 1]","[47, 16]","[17, 3]"
3,Argentina Nacional B,75,2 Estudiantes Rio Cuarto,1 - 1,Gimnasia Mendoza,"[2, 4]","[49, 47]","[5, 5]"
4,Bolivia Apertura,77,1 Real Tomayapo,2 - 1,Club Aurora 3,"[5, 7]","[28, 39]","[6, 15]"
5,Mauritania Cup,80,2 ASC Gendrim,0 - 0,Kaedi FC 1 1,"[3, 2]","[58, 61]","[9, 10]"
6,Argentina Nacional B,72,2 Club Atletico Temperley,1 - 0,Defensores Unidos 5 1,"[3, 3]","[74, 43]","[7, 4]"


In [58]:
def enviar_mensagem(msg):
  bot_token = '6252849643:AAESWkCUSB12DIkAKbbgKaSRLSz2TWpFzvI'
  chat_id = -1001803280902
  url = f'https://api.telegram.org/bot{bot_token}/sendMessage?chat_id={chat_id}&text={msg}'
  requests.post(url)

In [59]:
mensagens_enviadas = []

In [60]:

for i in range(len(df)):
    j = df.iloc[i]
    tempo = j['Cronometro']
    
    appm_home = round(j['Dangerous Attack'][0]/tempo,1)
    appm_away = round(j['Dangerous Attack'][1]/tempo,1)
    
    cg_home = j['Corner'][0]+ j ['Shots'][0]
    cg_away = j['Corner'][0]+ j ['Shots'][1]
    
    condicao_home = appm_home >=1 and cg_home >= 15
    condicao_away = appm_away >=1 and cg_away >= 15

    t1 = 0 < tempo < 45
    t2 = 45 < tempo < 90
    
    if (condicao_home or condicao_away) and (t1 or t2):
      if t1:
        t = 't1'
      else:
          t = 't2'
      home = re.sub(r'^\d{1} ','',j['Home'])
      away = re.sub(r'\d{1}$','',j['Away'])
      if f'{home} x {away}_{t}' not in mensagens_enviadas:
        msg = (f'''

  Liga:{j['League']}
  jogo: {home} x {away}
  Placar: {j['Score']}
  Appm: {appm_home} - {appm_away}
  CG: {cg_home} - {cg_away}
  ''')
      enviar_mensagem(msg)
      mensagens_enviadas.append(f'{home} x {away}_{t}')
      sleep(2)